Foi retirado vínculo do prof pois só tem o tipo 'Ativo Permanente'<br>
Foi retirado jornada de trabalho pois somente 3 turmas tinham profs com 20h semanais. Todo o resto é dedicação exclusiva.

In [2]:
import numpy as np
import pandas as pd

In [3]:
#Função que insere 'vazio' no campo null. Sem isso os dataframes iram quebrar.
def TratarDfVaziosProf(df):
    if(df.empty):
        df = pd.DataFrame()
        df = df.append({'id_servidor':0, 'nome':'vazio', 'formacao':'vazio', 'tipo_jornada_trabalho':'vazio', 'vinculo':'vazio', 'classe_funcional':'vazio', 'lotacao': 'vazio'}, ignore_index=True)
        return df
    return df

In [4]:
pd_disc = pd.read_csv('disciplinas.csv')
pd_docentes = pd.read_csv('docentes.csv', encoding = "utf-8", sep=";", usecols=['id_servidor', 'nome', 'formacao', 'lotacao'])
all_turma_matriculados = pd.read_csv('all_turma_matriculados.csv', encoding = "utf-8", sep=",")

In [ ]:
framesAux = []
for index_turma, row_turma in all_turma_matriculados.iterrows():
    pd_turmas = pd.read_csv(row_turma['csv_turma'], error_bad_lines = False, warn_bad_lines = False, encoding = "utf-8", sep=";", usecols=['id_turma', 'codigo_turma', 'id_docente_interno', 'id_componente_curricular', 'local', 'ano', 'periodo', 'descricao_horario', 'situacao_turma'])
    framesInter = []
    for index_pd_disc, row_pd_disc in pd_disc.iterrows():

        id_disc = row_pd_disc['id_disc']
        list_turmas = pd_turmas[pd_turmas['id_componente_curricular'] == id_disc] # filtra todas as turmas com o id da disciplina
        list_turmas_disc = list_turmas.assign(nome_disc=row_pd_disc['nome_disc']) # add uma coluna com o nome da disciplina

        listProf = []
        for index, row in list_turmas_disc.iterrows():
            prof = pd_docentes[pd_docentes['id_servidor'] == int(row["id_docente_interno"])]
            listProf.append(TratarDfVaziosProf(prof))

        df_profs = pd.concat(listProf)
        list_turmas_disc_prof = list_turmas_disc.assign(
            nome_prof = df_profs['nome'].tolist(),
            id_prof = df_profs['id_servidor'].tolist(),
            formacao_prof = df_profs['formacao'].tolist(),
            lotacao_prof = df_profs['lotacao'].tolist())
        framesInter.append(list_turmas_disc_prof)
    
    df_turma_professorAux = pd.concat(framesInter) # concatena todos os dataframes
    framesAux.append(df_turma_professorAux)
    
df_turma_professor = pd.concat(framesAux) 

In [ ]:
frames = []
ano_periodo_ant = ''
url_matriculados = ''
pd_alunos = 0

for turma in df_turma_professor['id_turma'].unique():
    row = df_turma_professor[df_turma_professor['id_turma'] == turma].values[0]
    ano = str(row[5])
    periodo = str(int(row[6]))
    ano_periodo = ano+"_"+periodo
    if(ano_periodo != ano_periodo_ant):
        url_matriculados = all_turma_matriculados[all_turma_matriculados['ano_semestre'] == ano_periodo]['csv_matriculados'].values[0]
        pd_alunos = pd.read_csv(url_matriculados, encoding = "utf-8", sep=";", usecols=['id_turma', 'discente', 'unidade', 'nota', 'reposicao', 'media_final', 'numero_total_faltas', 'descricao'])
        ano_periodo_ant = ano_periodo

    df_alunos = pd_alunos[pd_alunos['id_turma'] == int(turma)]
    frames.append(df_alunos)
    
df_alunos = pd.concat(frames) # concatena todos os dataframes
df_alunos.to_csv('turma_aluno.csv', encoding='UTF-8')

In [ ]:
listTotalAlunos = []
listAprovados = []
listReprovados = []
listMean = []
listMin = []
list1q = []
list2q = []
list3q = []
listmax = []

for turma in df_turma_professor['id_turma']:
    df_turma_alunos_3notas = df_alunos[df_alunos['id_turma'] == int(turma)] #nesse df tá com as 3 notas do aluno

    if(df_turma_alunos_3notas['discente'].size > 0):
        
        #Tem 3 linhas com 1ª, 2ª e 3º nota do aluno, eu retiro 2 linhas para o mesmo aluno não entrar na contagem
        df_turma_alunos_final = df_turma_alunos_3notas.drop_duplicates(subset=['discente']) 
        totalAlunos = len(df_turma_alunos_final.index)
        aprovados = len(df_turma_alunos_final[df_turma_alunos_final['media_final'] >= 5].index)
        reprovados = totalAlunos-aprovados
        
        media_final_summary = df_turma_alunos_final.describe()['media_final']
        
        listTotalAlunos.append(totalAlunos)
        listAprovados.append(aprovados)
        listReprovados.append(reprovados)
        listMean.append(format(media_final_summary['mean'], '.2f'))
        listMin.append(media_final_summary['min'])
        list1q.append(media_final_summary['25%'])
        list2q.append(media_final_summary['50%'])
        list3q.append(media_final_summary['75%'])
        listmax.append(media_final_summary['max'])

    else: #ocorre quando não encontra uma turma. Isso ocorre quando a turma foi excluida.
        listTotalAlunos.append(0)
        listAprovados.append(0)
        listReprovados.append(0)
        listMean.append(0)
        listMin.append(0)
        list1q.append(0)
        list2q.append(0)
        list3q.append(0)
        listmax.append(0)

df_turma_professor = df_turma_professor.assign(
    total_alunos = listTotalAlunos,
    aprovados = listAprovados,
    reprovados = listReprovados,
    disc_mean = listMean,
    disc_min = listMin,
    disc_q1 = list1q,
    disc_q2 = list2q,
    disc_q3 = list3q,
    disc_max = listmax
    )

In [ ]:
turno = []
ano_semestre = []
for index, row in df_turma_professor.iterrows():
    ano = str(row['ano'])
    periodo = str(int(row['periodo']))
    ano_periodo = ano+"_"+periodo
    ano_semestre.append(ano_periodo)
    
df_turma_professor = df_turma_professor.assign(ano_periodo = ano_semestre)
df_turma_professor.to_csv('turma_professor_notas.csv', encoding='UTF-8')


for index,row in df_turma_professor.iterrows():
    m_count = row['descricao_horario'].count('M')
    t_count = row['descricao_horario'].count('T')
    n_count = row['descricao_horario'].count('N')
    if(m_count > 0):
        df_turma_professor.loc[index,'Turno'] = 'M'
    elif(t_count > 0):
        df_turma_professor.loc[index,'Turno'] = 'T'
    elif(n_turma > 0):
        df_turma_professor.loc[index,'Turno'] = 'N'
    else:
        break
df_turma_professor

In [10]:
#pd.read_csv('turma_professor_notas.csv')


,Unnamed: 0,id_turma,codigo_turma,id_docente_interno,id_componente_curricular,local,ano,periodo,descricao_horario,situacao_turma,...,aprovados,disc_max,disc_mean,disc_min,disc_q1,disc_q2,disc_q3,reprovados,total_alunos,ano_periodo
0,955,1217917,1,5756224.0,55031.0,B203 no CIVT,2014,1.0,236M1234 (27/01/2014 - 14/06/2014),CONSOLIDADA,...,33,10.0,7.15,0.6,6.025,7.45,8.900,42,75,2014_1
1,956,1217917,1,5756386.0,55031.0,B203 no CIVT,2014,1.0,236M1234 (27/01/2014 - 14/06/2014),CONSOLIDADA,...,33,10.0,7.15,0.6,6.025,7.45,8.900,42,75,2014_1
2,957,1217918,2,5756188.0,55031.0,B204 no CIVT,2014,1.0,236M1234 (27/01/2014 - 14/06/2014),CONSOLIDADA,...,39,9.3,5.58,0.0,5.100,5.80,7.375,24,63,2014_1
3,958,1217918,2,5756386.0,55031.0,B204 no CIVT,2014,1.0,236M1234 (27/01/2014 - 14/06/2014),CONSOLIDADA,...,39,9.3,5.58,0.0,5.100,5.80,7.375,24,63,2014_1
4,959,1217919,3,5756208.0,55031.0,B203 no CIVT,2014,1.0,236N1234 (27/01/2014 - 14/06/2014),CONSOLIDADA,...,29,9.5,5.55,0.0,3.600,6.50,7.500,57,86,2014_1
5,960,1217919,3,5756386.0,55031.0,B203 no CIVT,2014,1.0,236N1234 (27/01/2014 - 14/06/2014),CONSOLIDADA,...,29,9.5,5.55,0.0,3.600,6.50,7.500,57,86,2014_1
6,961,1217920,4,5756213.0,55031.0,B204 no CIVT,2014,1.0,236N1234 (27/01/2014 - 14/06/2014),CONSOLIDADA,...,27,9.5,4.59,0.0,1.600,5.50,6.650,41,68,2014_1
7,962,1217920,4,5755819.0,55031.0,B204 no CIVT,2014,1.0,236N1234 (27/01/2014 - 14/06/2014),CONSOLIDADA,...,27,9.5,4.59,0.0,1.600,5.50,6.650,41,68,2014_1
8,997,1217943,1,5755872.0,52662.0,A306 no CIVT,2014,1.0,246M56 (27/01/2014 - 14/06/2014),CONSOLIDADA,...,13,10.0,4.20,0.0,0.900,5.00,7.700,16,29,2014_1
9,998,1217944,2,5755872.0,52662.0,A308 no CIVT,2014,1.0,246N12 (27/01/2014 - 14/06/2014),CONSOLIDADA,...,3,8.3,3.18,0.4,1.250,2.40,4.800,10,13,2014_1
